# Scheduler K8S Test

Assumes you have a Kind cluster running with metallb. No other dependencies required.
You can use [Seldon Ansible Kind Playbook](https://github.com/SeldonIO/ansible-k8s-collection/blob/master/playbooks/kind.yaml)

## Setup

* `make kind-image-install-all`
* `make deploy` 


In [26]:
SCHEDULER_IP=!kubectl get svc seldon-scheduler -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
SCHEDULER_IP=SCHEDULER_IP[0]
import os
os.environ['SCHEDULER_IP'] = SCHEDULER_IP

In [27]:
MESH_IP=!kubectl get svc seldon-mesh -n seldon-mesh -o jsonpath='{.status.loadBalancer.ingress[0].ip}'
MESH_IP=MESH_IP[0]
import os
os.environ['MESH_IP'] = MESH_IP

## No Auth Example

In [28]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":500},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [29]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-01-27T19:33:51.552435682Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-01-27T19:33:51.552435682Z"
      }
    }
  ]
}


In [30]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55ab9807c4f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55ab9807c4f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Thu, 27 Jan 2022 19:33:53 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1347
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"776563b6-2a3c-4ca6-9a60-d52c94230125","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [31]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [32]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [33]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55c6ad8364f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55c6ad8364f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)








* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


In [24]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Unloaded",
          "lastChangeTimestamp": "2022-01-27T17:29:36.535712038Z"
        }
      },
      "state": {
        "state": "ModelTerminated",
        "lastChangeTimestamp": "2022-01-27T17:29:36.535712038Z"
      }
    }
  ],
  "deleted": true
}


In [25]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "1000000000"
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}


## Auth Preparation - Prepare Minio with Iris Model

Install minio in a kind cluster with Ansible.

Create rclone.conf changing the ip address for minio as appropriate

```
[s3]
type = s3
provider = minio
env_auth = false
access_key_id = minioadmin
secret_access_key = minioadmin
endpoint = http://172.18.255.1:9000
```
 Download iris model
 
 ```
 cd /tmp
 gsutil cp -r gs://seldon-models/mlserver/iris .
 ```
  
 Copy iris model to minio
 
 ```
 rclone --config ./rclone.conf copy iris s3://models/iris
 ```

## Inline RClone Config Example


In [63]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"},\
            modelSpec: {uri:"s3://models/iris",\
                        storageConfig: { \
                        storageRcloneConfig: "{\"type\":\"s3\",\"name\":\"s3\",\"parameters\":{\"provider\":\"minio\",\"env_auth\":\"false\",\"access_key_id\":\"minioadmin\",\"secret_access_key\":\"minioadmin\",\"endpoint\":\"http://minio.minio-system:9000\"}}" \
     } \
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [65]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-01-24T19:00:33.975552993Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-01-24T19:00:33.975552993Z"
      }
    }
  ]
}


In [66]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "999999500",
      "numLoadedModels": 1
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}


In [67]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55a26decd4f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55a26decd4f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Mon, 24 Jan 2022 19:00:52 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1008
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"6e44914b-5325-4f2a-a420-7d53604a0e6d","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [68]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [69]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


## K8S Secret RClone Config Example

 * Update endpoint for minio below

In [70]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://minio.minio-system:9000

Overwriting minio-secret.yaml


In [71]:
!kubectl apply -f minio-secret.yaml

secret/minio-secret created


In [72]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"\
                    kubernetesMeta: {namespace: "seldon-mesh"}},\
            modelSpec: {uri:"s3://models/iris",\
                        storageConfig: { \
                          storageSecretName: "minio-secret" \
                         } \
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [73]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "999999500",
      "numLoadedModels": 1
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}


In [74]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x5638b12024f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x5638b12024f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Mon, 24 Jan 2022 19:01:33 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1048
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"d0b58d58-f9c4-4512-978b-32f85447ee3f","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [75]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [76]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


## Using Agent ConfigMap

 * Edit the AgentConfig map to include the `minio-secret` as one of the rclone defaults, e.g.:
 
 ```
 apiVersion: v1
kind: ConfigMap
metadata:
  name: seldon-agent
data:
  agent.json: |-
    {
       "rclone" : {
           "config_secrets": ["seldon-rclone-gs-public","minio-secret"]
       },
    }

 ```

In [86]:
%%writefile minio-secret.yaml
apiVersion: v1
kind: Secret
metadata:
  name: minio-secret
  namespace: seldon-mesh
type: Opaque
stringData:
  s3: |
    type: s3
    name: s3
    parameters:
      provider: minio
      env_auth: false
      access_key_id: minioadmin
      secret_access_key: minioadmin
      endpoint: http://minio.minio-system:9000

Overwriting minio-secret.yaml


In [87]:
!kubectl apply -f minio-secret.yaml

secret/minio-secret configured


In [92]:
!grpcurl --format text -d '\
         model { \
            meta: { name:"iris"\
                    kubernetesMeta: {namespace: "seldon-mesh"}},\
            modelSpec: {uri:"s3://models/iris",\
                        requirements:["sklearn"],\
                        memoryBytes:500},\
            deploymentSpec:{replicas:1}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto   ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

In [93]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "kubernetesMeta": {
        "namespace": "seldon-mesh"
      },
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-01-24T19:06:30.411001720Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-01-24T19:06:30.411001720Z"
      }
    }
  ]
}


In [94]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "999999500",
      "numLoadedModels": 1
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}


In [95]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55c1f2ed04f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55c1f2ed04f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Mon, 24 Jan 2022 19:06:44 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1056
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"8531e03d-7fac-44b7-94ed-195f13bbaf6d","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [96]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [97]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


# Versions Test

In [98]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":50000},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [99]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-01-24T19:06:59.662422552Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-01-24T19:06:59.662422552Z"
      }
    }
  ]
}


In [100]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x564fb29c44f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x564fb29c44f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Mon, 24 Jan 2022 19:07:09 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1076
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"1","id":"ec993354-c708-4236-ac94-ba5e145d6700","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [102]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "1",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [103]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "999950000",
      "numLoadedModels": 1
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "numLoadedModelReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}


In [104]:
!grpcurl -d '{"model":{ \
              "meta":{"name":"iris"},\
              "modelSpec":{"uri":"gs://seldon-models/mlserver/iris",\
                           "requirements":["sklearn"],\
                           "memoryBytes":20000},\
              "deploymentSpec":{"replicas":1}}}' \
         -plaintext \
         -import-path ../../apis \
         -proto ../../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/LoadModel

{
  
}


In [105]:
!grpcurl -d '{"model":{"name":"iris"},"allVersions":true}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Unloaded",
          "lastChangeTimestamp": "2022-01-24T19:07:33.232903757Z"
        }
      },
      "state": {
        "state": "ModelTerminated",
        "lastChangeTimestamp": "2022-01-24T19:07:33.232903757Z"
      }
    },
    {
      "version": 2,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Available",
          "lastChangeTimestamp": "2022-01-24T19:07:33.227221974Z"
        }
      },
      "state": {
        "state": "ModelAvailable",
        "availableReplicas": 1,
        "lastChangeTimestamp": "2022-01-24T19:07:33.227221974Z"
      }
    }
  ]
}


In [106]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x55f3f000c4f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x55f3f000c4f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)
> POST /v2/models/iris/infer HTTP/1.1
> Host: 172.18.255.3
> User-Agent: curl/7.64.0
> Accept: */*
> Content-Type: application/json
> seldon-model: iris
> Content-Length: 94
> 
* upload completely sent off: 94 out of 94 bytes
< HTTP/1.1 200 OK
< date: Mon, 24 Jan 2022 19:07:45 GMT
< server: envoy
< content-length: 194
< content-type: application/json
< x-envoy-upstream-service-time: 1007
< 
* Connection #0 to host 172.18.255.3 left intact
{"model_name":"iris","model_version":"2","id":"7d6ca29e-73f5-434d-ab8c-87f0596ac6a4","parameters":null,"outputs":[{"name":"predict","shape":[1],"datatype":"INT64","parameters":null,"data":[2]}]}

In [107]:
!cd ../v2 && \
    grpcurl -d '{"model_name":"iris","inputs":[{"name":"input","contents":{"fp32_contents":[1,2,3,4]},"datatype":"FP32","shape":[1,4]}]}' \
        -plaintext \
        -proto grpc_service.proto \
        -rpc-header seldon-model:iris \
       ${MESH_IP}:80 inference.GRPCInferenceService/ModelInfer

{
  "modelName": "iris",
  "modelVersion": "2",
  "outputs": [
    {
      "name": "predict",
      "datatype": "INT64",
      "shape": [
        "1"
      ],
      "contents": {
        "int64Contents": [
          "2"
        ]
      }
    }
  ]
}


In [108]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "999980000"
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}


In [109]:
!grpcurl -d '{"model":{"name":"iris"}}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/UnloadModel

{
  
}


In [110]:
!curl -v http://${MESH_IP}/v2/models/iris/infer -H "Content-Type: application/json" -H "seldon-model: iris"\
        -d '{"inputs": [{"name": "predict", "shape": [1, 4], "datatype": "FP32", "data": [[1, 2, 3, 4]]}]}'

* Expire in 0 ms for 6 (transfer 0x5633526574f0)
*   Trying 172.18.255.3...
* TCP_NODELAY set
* Expire in 200 ms for 4 (transfer 0x5633526574f0)
* Connected to 172.18.255.3 (172.18.255.3) port 80 (#0)








* upload completely sent off: 94 out of 94 bytes






* Closing connection 0


In [111]:
!grpcurl -d '{"model":{"name":"iris"},"allVersions":true}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ModelStatus

{
  "modelName": "iris",
  "versions": [
    {
      "version": 1,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Unloaded",
          "lastChangeTimestamp": "2022-01-24T19:07:33.232903757Z"
        }
      },
      "state": {
        "state": "ModelTerminated",
        "lastChangeTimestamp": "2022-01-24T19:07:33.232903757Z"
      }
    },
    {
      "version": 2,
      "serverName": "mlserver",
      "modelReplicaState": {
        "0": {
          "state": "Unloaded",
          "lastChangeTimestamp": "2022-01-24T19:08:10.046278335Z"
        }
      },
      "state": {
        "state": "ModelTerminated",
        "lastChangeTimestamp": "2022-01-24T19:08:10.046278335Z"
      }
    }
  ],
  "deleted": true
}


In [112]:
!grpcurl -d '{"name":"mlserver"}' \
         -plaintext \
         -proto ../apis/mlops/scheduler/scheduler.proto  ${SCHEDULER_IP}:9004 seldon.mlops.scheduler.Scheduler/ServerStatus

{
  "serverName": "mlserver",
  "resources": [
    {
      "totalMemoryBytes": "1000000000",
      "availableMemoryBytes": "1000000000"
    }
  ],
  "expectedReplicas": 1,
  "availableReplicas": 1,
  "kubernetesMeta": {
    "namespace": "seldon-mesh",
    "generation": "1"
  }
}
